In [16]:
from dotenv import load_dotenv
from agents import Agent, trace, function_tool, Runner
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import resend
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio

In [7]:
load_dotenv(override=True)

True

In [8]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [9]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

sales_agent2 = Agent(
    name="Engaging Sales Agent", instructions=instructions2, model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    ),
)

sales_agent3 = Agent(
    name="Busy Sales Agent", instructions=instructions3, model=LitellmModel(
        model="gemini/gemini-2.0-flash", api_key=os.getenv("GEMINI_API_KEY")
    )
)

In [15]:
result = Runner.run_streamed(sales_agent1,input="Write a cold email to a company that needs to prepare for a SOC2 audit.")
async for event in result.stream_events():
    if event.type =="raw_response_event" and isinstance(event.data,ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC2 Compliance with AI-Powered Efficiency

Dear [Contact Name],

My: Streamline Your SOC2 Compliance with AI-Powered Efficiency

Dear [Contact Name],

My name is [Your Name] and I'm a Sales Agent at ComplAI, a company focused on simplifying the SOC2 compliance process. I'm reaching out to you at [Company Name] because I understand you may be preparing for a SOC2 audit, and I believe ComplAI could significantly streamline your efforts.

Preparing for a SOC2 audit can be a complex and resource-intensive undertaking. From gathering evidence and managing documentation to ensuring continuous compliance, it demands significant time and expertise.

ComplAI offers an AI-powered SaaS solution that automates many of these crucial tasks, allowing your team to focus on core business objectives. Our platform helps you:

*   **Automate Evidence Collection:** Seamlessly integrate with your existing systems to automatically gather and map evidence to SOC2 requirements.
*   **

In [17]:
message = "Write a cold email to a company that needs to prepare for a SOC2 audit."

with trace("Parallel Cold Emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message),
    )


outputs = [result.final_output for result in results]
for output in outputs:
    print(output, "\n\n---\n\n")


Subject: Streamline Your SOC2 Compliance with ComplAI

Dear [Contact Name],

My name is [Your Name] and I'm a Sales Agent at ComplAI, a company helping businesses like yours simplify and expedite the often complex process of achieving SOC2 compliance.

I understand that preparing for a SOC2 audit can be a resource-intensive undertaking, demanding significant time and effort from your team. Ensuring all required controls are in place, documenting evidence, and undergoing the audit itself can be a drain on productivity and potentially delay critical business objectives.

ComplAI offers a SaaS platform powered by AI that streamlines your SOC2 journey. Our solution helps you:

*   **Automate evidence collection:** Seamlessly gather and organize the necessary evidence to demonstrate compliance with SOC2 requirements.
*   **Identify compliance gaps:** Pinpoint areas where your current security posture needs improvement to meet SOC2 standards.
*   **Simplify policy creation:** Utilize pre-bui